## This is to preprocess the kaggle data and split it into train, val, test splits with appropriate target (y) variables

In [1]:
import pandas as pd
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

### Set up dataset paths

In [2]:
# To get this working, download the interest, gold and BTC data from the following links:
# BTC: https://www.kaggle.com/datasets/prasoonkottarathil/btcinusd?fbclid=IwZXh0bgNhZW0CMTEAAR1nQkLa0WBz1xAmjoehJoNypO13dIIHXqq1lGqC-OQzINtcfUUIdZM1PwE_aem_5o1oGQ7Q57RLLYCH2O3zjw
# Interest: https://www.kaggle.com/datasets/federalreserve/interest-rates
# Gold: https://www.kaggle.com/datasets/rizkykiky/gold-price-dataset
# Make sure to put the files in the same directory as this script or adjust the paths accordingly.

BTC_dirname = "bitcoin_some"
INTEREST_dirname = "interest_rates"
GOLD_dirname = "gold_price"

BTC_filename = "BTC-Daily.csv"
INTEREST_file = "index.csv"
GOLD_file = "Daily.csv"

BTC_path = os.path.join(BTC_dirname, BTC_filename)
INTEREST_path = os.path.join(INTEREST_dirname, INTEREST_file)
GOLD_path = os.path.join(GOLD_dirname, GOLD_file)

### Read data

In [3]:
btc_df = pd.read_csv(BTC_path, parse_dates=["date"])
btc_df = btc_df.sort_values(by="date")
print(f"length of BTC_df: {len(btc_df)}")
print(btc_df.head())
print(btc_df.tail())

length of BTC_df: 2651
            unix       date   symbol    open    high     low   close  \
2650  1417132800 2014-11-28  BTC/USD  363.59  381.34  360.57  376.28   
2649  1417219200 2014-11-29  BTC/USD  376.42  386.60  372.25  376.72   
2648  1417305600 2014-11-30  BTC/USD  376.57  381.99  373.32  373.34   
2647  1417392000 2014-12-01  BTC/USD  376.40  382.31  373.03  378.39   
2646  1417478400 2014-12-02  BTC/USD  378.39  382.86  375.23  379.25   

      Volume BTC  Volume USD  
2650  3220878.18     8617.15  
2649  2746157.05     7245.19  
2648  1145566.61     3046.33  
2647  2520662.37     6660.56  
2646  2593576.46     6832.53  
         unix       date   symbol      open      high       low     close  \
4  1645747200 2022-02-25  BTC/USD  38360.93  39727.97  38027.61  39231.64   
3  1645833600 2022-02-26  BTC/USD  39242.64  40330.99  38600.00  39146.66   
2  1645920000 2022-02-27  BTC/USD  39146.66  39886.92  37015.74  37712.68   
1  1646006400 2022-02-28  BTC/USD  37717.10  44256

In [4]:
# interest file has separate columns for "Year", "Month", "Day", need to combine them into one date column
interest_df = pd.read_csv(INTEREST_path)
interest_df["Year_Month_Day"] = pd.to_datetime(interest_df[["Year", "Month", "Day"]])
interest_df.rename(columns={"Year_Month_Day": "date"}, inplace=True)
interest_df.drop(columns=["Year", "Month", "Day"], inplace=True)
interest_df = interest_df.sort_values('date')
interest_df = interest_df[["date", "Effective Federal Funds Rate", "Real GDP (Percent Change)", "Unemployment Rate", "Inflation Rate"]]
interest_df.rename(columns={"Effective Federal Funds Rate": "interest_rate",
                       "Real GDP": "gdp", "Unemployment Rate": "unemployment_rate",
                       "Inflation Rate": "inflation_rate"}, inplace=True)

# replace nans with the most recent value
interest_df.fillna(method='ffill', inplace=True)
# remove all rows with date before 2014-09-01
interest_df = interest_df[interest_df["date"] >= "2014-09-01"]
# remove all rows with date after 2022-03-01
interest_df = interest_df[interest_df["date"] <= "2022-03-01"]
# reset index
interest_df.reset_index(drop=True, inplace=True)

print(interest_df.head())

        date  interest_rate  Real GDP (Percent Change)  unemployment_rate  \
0 2014-09-01           0.09                        5.0                5.9   
1 2014-10-01           0.09                        2.3                5.7   
2 2014-11-01           0.09                        2.3                5.8   
3 2014-12-01           0.12                        2.3                5.6   
4 2015-01-01           0.11                        2.0                5.7   

   inflation_rate  
0             1.7  
1             1.8  
2             1.7  
3             1.6  
4             1.6  


C:\Users\Ethan\AppData\Local\Temp\ipykernel_42848\1954680175.py:13: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  interest_df.fillna(method='ffill', inplace=True)


In [5]:
gold_df = pd.read_csv(GOLD_path, parse_dates=["Date"])
gold_df.rename(columns={"Date": "date"}, inplace=True)
gold_df = gold_df.sort_values('date')
gold_df = gold_df[["date", "USD"]] # keep only the date and USD columns to make things simple
gold_df.rename(columns={"USD": "gold_price"}, inplace=True)
gold_df['gold_price'] = gold_df['gold_price'].str.replace(',', '').astype(float)


# replace nans with the most recent value
gold_df.fillna(method='ffill', inplace=True)
# remove all rows with date before 2014-09-01
gold_df = gold_df[gold_df["date"] >= "2014-09-01"]
# remove all rows with date after 2022-03-01
gold_df = gold_df[gold_df["date"] <= "2022-03-01"]
# reset index
gold_df.reset_index(drop=True, inplace=True)
print(gold_df.head())
print(gold_df["gold_price"])

        date  gold_price
0 2014-09-01      1286.5
1 2014-09-02      1267.0
2 2014-09-03      1265.5
3 2014-09-04      1271.5
4 2014-09-05      1266.0
0       1286.5
1       1267.0
2       1265.5
3       1271.5
4       1266.0
         ...  
1952    1904.7
1953    1936.3
1954    1884.8
1955    1909.9
1956    1922.0
Name: gold_price, Length: 1957, dtype: float64


C:\Users\Ethan\AppData\Local\Temp\ipykernel_42848\4105426102.py:10: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  gold_df.fillna(method='ffill', inplace=True)


### Merge dataframes

In [6]:
# now we merge the dataframes onto the btc_df dataframe where values will be the most recent value before the date based on the date column
market_df = pd.merge_asof(
    btc_df,
    interest_df,
    on='date',
    direction='backward'
)

market_df = pd.merge_asof(
    market_df,
    gold_df,
    on='date',
    direction='backward'
)

market_df.drop(columns=["unix", "symbol"], inplace=True) # unix is not needed because we have date, symbol is the same for all rows
print(market_df.head())


        date    open    high     low   close  Volume BTC  Volume USD  \
0 2014-11-28  363.59  381.34  360.57  376.28  3220878.18     8617.15   
1 2014-11-29  376.42  386.60  372.25  376.72  2746157.05     7245.19   
2 2014-11-30  376.57  381.99  373.32  373.34  1145566.61     3046.33   
3 2014-12-01  376.40  382.31  373.03  378.39  2520662.37     6660.56   
4 2014-12-02  378.39  382.86  375.23  379.25  2593576.46     6832.53   

   interest_rate  Real GDP (Percent Change)  unemployment_rate  \
0           0.09                        2.3                5.8   
1           0.09                        2.3                5.8   
2           0.09                        2.3                5.8   
3           0.12                        2.3                5.6   
4           0.12                        2.3                5.6   

   inflation_rate  gold_price  
0             1.7      1182.8  
1             1.7      1182.8  
2             1.7      1182.8  
3             1.6      1194.0  
4         

In [7]:
# # turn 'open', 'close', 'high', 'low', 'Volume BTC', 'Volume USD', 'gold_price' columns into percentage change from last day
market_df['open'] = market_df['open'].pct_change()
# market_df['close'] = market_df['close'].pct_change()
# market_df['high'] = market_df['high'].pct_change()
# market_df['low'] = market_df['low'].pct_change()
# market_df['Volume BTC'] = market_df['Volume BTC'].pct_change()
# market_df['Volume USD'] = market_df['Volume USD'].pct_change()
# market_df['gold_price'] = market_df['gold_price'].pct_change()

market_df['open'] = market_df['open'].fillna(0)
# market_df['close'] = market_df['close'].fillna(0)
# market_df['high'] = market_df['high'].fillna(0)
# market_df['low'] = market_df['low'].fillna(0)
# market_df['Volume BTC'] = market_df['Volume BTC'].fillna(0)
# market_df['Volume USD'] = market_df['Volume USD'].fillna(0)
# market_df['gold_price'] = market_df['gold_price'].fillna(0)

# date = market_df['date']
# market_df.drop(columns=["date"], inplace=True) # drop date column for now, we will add it back later

# mean = market_df.mean(axis=0)
# std = market_df.std(axis=0)
# market_df = (market_df - mean) / std # standardize the data

# market_df['date'] = date # add date column back


In [8]:
# save the dataframe to a csv file
market_df.to_csv("market_data_V1.csv", index=False)
